In [30]:
# Imports
import pandas as pd
import numpy as np

In [31]:
df=pd.read_excel('C:/Users/ip 5/Downloads/Copy of GSP_DATA_19_22(208).xlsx')

In [32]:
df_no_nul=df.dropna()

In [33]:
date_1=df_no_nul.pop('DATETIME')

In [34]:
date_2=pd.to_datetime(date_1)

In [35]:
date_2

0       2019-11-01 06:00:00
1       2019-11-01 07:00:00
2       2019-11-01 08:00:00
3       2019-11-01 09:00:00
4       2019-11-01 10:00:00
                ...        
13645   2022-10-31 14:00:00
13646   2022-10-31 15:00:00
13647   2022-10-31 16:00:00
13648   2022-10-31 17:00:00
13649   2022-10-31 18:00:00
Name: DATETIME, Length: 13638, dtype: datetime64[ns]

In [36]:
Month=pd.DataFrame(date_2.dt.month).rename(columns = {'DATETIME': 'MONTH'}, inplace = False)
Day = pd.DataFrame(date_2.dt.day).rename(columns = {'DATETIME': 'DAY'}, inplace = False)
Hour=pd.DataFrame(date_2.dt.hour).rename(columns = {'DATETIME': 'HOUR'}, inplace = False)
Monthday=pd.DataFrame((date_2.dt.month+date_2.dt.day)).rename(columns = {'DATETIME': 'MONTHDAY'}, inplace = False)
Yearday=pd.DataFrame(((date_2.dt.month)-1)*30.4+(date_2.dt.day)).rename(columns = {'DATETIME': 'YEARDAY'}, inplace = False)
df3=pd.concat([date_2,Monthday,Yearday,Month,Day,Hour,df_no_nul],axis=1)
print('Raw Data :\n',display(df3.head()))

,DATETIME,MONTHDAY,YEARDAY,MONTH,DAY,HOUR,GHI(W/m2),DHI(W/m2),DNI(W/m2),TEMPERATURE,R.HUMIDITY,WINDSPEED,CABLE#1,CABLE#2,CABLE#3,CABLE#4,CABLE#5,TX(LV)
0,2019-11-01 06:00:00,12,305.0,11,1,6,0.0,0.0,1.0,23.2,95.26,11.9,0.00,0,0.00,0.00,0.00,0.00
1,2019-11-01 07:00:00,12,305.0,11,1,7,261.0,145.0,436.0,24.3,89.70,17.6,2.68,2.74,2.68,2.75,1.77,12.16
2,2019-11-01 08:00:00,12,305.0,11,1,8,123.0,127.0,5.0,25.8,80.06,22.7,4.68,3.03,2.71,1.86,0.88,12.52
3,2019-11-01 09:00:00,12,305.0,11,1,9,775.0,205.0,748.0,28.2,69.53,22.3,6.64,7.77,7.47,7.93,4.62,35.92
4,2019-11-01 10:00:00,12,305.0,11,1,10,936.0,129.0,870.0,28.3,66.23,23.8,8.77,8.87,8.80,8.86,5.76,41.68


Raw Data :
 None


In [37]:
df4=df_no_nul[(df3.HOUR >= 6) & (df3.HOUR <= 17)]
df4.to_csv('sorted1.csv')
print('Filtered data :\n', display(df4.head()))

,GHI(W/m2),DHI(W/m2),DNI(W/m2),TEMPERATURE,R.HUMIDITY,WINDSPEED,CABLE#1,CABLE#2,CABLE#3,CABLE#4,CABLE#5,TX(LV)
0,0.0,0.0,1.0,23.2,95.26,11.9,0.00,0,0.00,0.00,0.00,0.00
1,261.0,145.0,436.0,24.3,89.70,17.6,2.68,2.74,2.68,2.75,1.77,12.16
2,123.0,127.0,5.0,25.8,80.06,22.7,4.68,3.03,2.71,1.86,0.88,12.52
3,775.0,205.0,748.0,28.2,69.53,22.3,6.64,7.77,7.47,7.93,4.62,35.92
4,936.0,129.0,870.0,28.3,66.23,23.8,8.77,8.87,8.80,8.86,5.76,41.68


Filtered data :
 None


In [38]:

from sklearn.model_selection import train_test_split
X=df3.iloc[:,1:12]
y=df3.iloc[:,-1]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


In [39]:
#Removing -ve values
y[y< 0] = 0
y[y< 0] = 0
X[X< 0] = 0
X[X< 0] = 0

C:\Users\ip 5\AppData\Local\Temp\ipykernel_320\3930281860.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y< 0] = 0
C:\Users\ip 5\AppData\Local\Temp\ipykernel_320\3930281860.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y< 0] = 0


In [40]:
y

0         0.000000
1        12.160000
2        12.520000
3        35.920000
4        41.680000
           ...    
13645    22.770552
13646     8.462348
13647     8.320916
13648     4.455108
13649     1.320000
Name: TX(LV), Length: 13638, dtype: float64

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
#Scaling X data 
scaler = StandardScaler()
df45 = scaler.fit_transform(X)
X_train_scaled =pd.DataFrame(df45)
scaler_test = StandardScaler()
df50 = scaler_test.fit_transform(X_test)
X_test_scaled =pd.DataFrame(df50)

In [42]:
#X variables scaled converted to float32 data type
Xn=np.asarray(X_train_scaled).astype(np.float32)
Xn_test=np.asarray(X_test_scaled).astype(np.float32)
#Y1 variables converted to float32 data type
y1_train =pd.DataFrame(y)
Y1_train = np.asarray(y1_train).astype(np.float32)
y1_test = pd.DataFrame(y_test)
Y1_test = np.asarray(y1_test).astype(np.float32)

In [43]:
#Solar Power Prediction using different ML techniques
#using Regression
X1 = X_train_scaled
X1_test =X_test_scaled
Y1 = Y1_train #Energy output

In [44]:
#Linear Regression algorithm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
lr = LinearRegression()
lr.fit(X1,Y1)
#Model Evaluation
Ypred_lr = lr.predict(X1_test)
Rsquare_lr=r2_score(Y1_test, Ypred_lr)
MAE_lr = mean_absolute_error(Y1_test,Ypred_lr)
RMSE_lr = np.sqrt(mean_squared_error(Y1_test, Ypred_lr))
Accuracy_lr = ((54.6-RMSE_lr)/54.6)*100
print("Rsquare_lr:",Rsquare_lr)
print("MAE_lr :",MAE_lr)
print("RMSE_lr:",RMSE_lr)
print("Accuracy:",Accuracy_lr )
#=====================================


Rsquare_lr: -5.773141201204404e+19
MAE_lr : 88123507481.08661
RMSE_lr: 106597175444.79529
Accuracy: -195232921960.06464
